In [ ]:
!pip install mediapipe
!pip install py7zr
!pip install tensorflow==2.4.0

In [1]:
from pathlib import Path
from csv import writer
from numpy import array, zeros, concatenate, ndarray, expand_dims
import mediapipe as mp
from shutil import move, rmtree
import cv2, os, random, pickle, py7zr
import pandas as pd
from functools import reduce 
from tensorflow.keras.models import load_model
from cv2 import VideoCapture, waitKey, cvtColor, flip, COLOR_BGR2RGB, COLOR_RGB2BGR, VideoWriter_fourcc, VideoWriter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
from os.path import split
from re import findall
from time import sleep

In [2]:
class ImageHandler:
    def draw_results(self, image, results):
        image.flags.writeable = True
        # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        return image


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
im = ImageHandler()
QR = []         #list of frames
model = "/content/drive/MyDrive/20frame_17classmodel/model36"
model = load_model(str(model))
classes = ['Address', 'Age', 'Class', 'How', 'Movie', 'Name', 'Phone', 'Play', 'Please', 'ThankYou', 'Time', 'What', 'When', 'Where', 'Work', 'You', 'Your']

In [4]:
def extract_landmarks(results):
        result_landmarks = {
            'face': {
                'landmark': results.face_landmarks,
                'shape': (468,)
            },
            'left hand': {
                'landmark': results.left_hand_landmarks,
                'shape': (21,)
            },
            'right hand': {
                'landmark': results.right_hand_landmarks,
                'shape': (21,)
            },
            'pose': {
                'landmark': results.pose_landmarks,
                'shape': (33,)
            }
        }
        result_all = []
        result_temp = []
        new_result_temp = []
        new_result_all = []
        new_new_result_temp = []
        for key, result in result_landmarks.items():
            if result['landmark']:
                new_result_temp = []
                new_new_result_temp = []
                result_temp = array([array([l.x, l.y, l.z]) for l in result['landmark'].landmark])

                for l in result['landmark'].landmark:
                    new_result_temp.append([l.x,l.y,l.z])
                for item in new_result_temp:
                    for temp_item in item:
                        new_new_result_temp.append(temp_item)
                
                result_temp = result_temp.flatten()
            else:
                result_temp = zeros((result['shape'][0] * 3, ))
                # print(result['shape'][0])
                shape = int(result['shape'][0]) * 3
                
                listofzeros = [0] * shape
                new_new_result_temp = listofzeros

            assert(result_temp.shape == (result['shape'][0] * 3, ))
            result_all.append(result_temp)
            new_result_all.append(new_new_result_temp)

        # print("OLDMETHOD",len(result_all),len(result_all[0]),result_all)
        # print("NEWMETHOD",len(new_result_all),len(new_result_all[0]),new_result_all)
        shape = 0
        result_final = array([])
        new_result_final = []
        for result in result_all:
            shape += result.shape[0]
            result_final = concatenate((result_final, result), axis=0)
        for item in new_result_all:
            for new_item in item:
                new_result_final.append(new_item)

        assert(result_final.shape == (shape,))
        # print(len(result_final),result_final)
        # print(len(new_result_final),new_result_final)
        return result_final

In [7]:
cap = VideoCapture(str('/content/432369.avi'))
frame_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))        
with mp_holistic.Holistic(
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
) as holistic:
    while cap.isOpened() and frame_length!=0:
        image: ndarray
        success, image = cap.read()
        if not success: continue
        image = flip(image, 1)
        image = cvtColor(image, COLOR_BGR2RGB)
        image.flags.writeable = False
        results = holistic.process(image)
        image = cvtColor(image, COLOR_RGB2BGR)
        image = im.draw_results(image, results)
        landmarks = extract_landmarks(results)
        # print("stopppping")
        QR.append(landmarks)
        frame_length-=1
random_number = random.randint(1,20)
random_number = 4
print(QR,len(QR))
QR_vid = expand_dims(array(QR[random_number:random_number+30]), 0)
print(QR_vid,len(QR_vid),QR_vid.shape)
if QR_vid.shape == (1,30,1629):
    # print(QR_vid)
    results = model.predict(QR_vid)
    # print(results)
    results = array(results).mean(axis=0)
    print(results)
    predicted_kls = classes[results.argmax(axis=0)]
    print("\n" + "Predicted class : " + predicted_kls + "   " +str(random_number)+":"+str(random_number+30))


[array([ 0.69105756,  0.30559638, -0.02698399, ...,  0.58738148,
        2.340518  ,  0.10588159]), array([ 0.68716443,  0.30429098, -0.02771606, ...,  0.59217149,
        2.35190916, -0.14965603]), array([ 0.67982233,  0.30492899, -0.02801149, ...,  0.58675343,
        2.32633591, -0.22153668]), array([ 0.67759532,  0.30685344, -0.02785609, ...,  0.60873258,
        2.33668399, -0.1387402 ]), array([ 0.67590612,  0.30847222, -0.02693951, ...,  0.60656226,
        2.33845782, -0.23685727]), array([ 0.67913687,  0.31074005, -0.02618725, ...,  0.58997124,
        2.30960965, -0.19781706]), array([ 0.68122947,  0.31084979, -0.02621309, ...,  0.56089705,
        2.30708599, -0.54017031]), array([ 0.68293327,  0.31139055, -0.02642986, ...,  0.57898337,
        2.31052852, -0.50301844]), array([ 0.68519521,  0.31560472, -0.02551622, ...,  0.57826936,
        2.22199464, -0.41267246]), array([ 0.68673402,  0.31863299, -0.02454635, ...,  0.58542264,
        2.21481943, -0.11519281]), array([ 0

In [ ]:
[9.1052840e-05 2.7475148e-01 9.8983182e-06 5.3240706e-06 5.8405003e-05
 1.3777944e-04 9.5911892e-06 1.6922007e-05 1.7198402e-04 7.2360384e-01
 1.3717686e-04 2.6242506e-05 2.6202169e-05 2.4063284e-04 1.8457959e-05
 5.2241696e-04 1.7261400e-04]

Predicted class : ThankYou   7:37

In [ ]:
FACE
[[ 0.34012839  0.39028478 -0.02805694]
 [ 0.33936661  0.35464504 -0.03737018]
 [ 0.33922029  0.36931205 -0.02309948]
 ...
 [ 0.34654242  0.31665885  0.00560285]
 [ 0.38319719  0.31148958  0.02210454]
 [ 0.38656992  0.30880308  0.02275069]]
FLATTEN FACE
[ 0.34012839  0.39028478 -0.02805694 ...  0.38656992  0.30880308
  0.02275069]

SINCE LEFT HAND EMPTY PUT ZERO
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

RIGHT HAND
[[ 2.81015396e-01  6.84284389e-01  2.25725302e-07]
 [ 3.25042158e-01  6.32034183e-01 -5.77157876e-03]
 [ 3.48924816e-01  5.87240875e-01 -1.95303522e-02]
 [ 3.74168038e-01  5.62958658e-01 -3.58712152e-02]
 [ 4.00897473e-01  5.43651700e-01 -5.09248823e-02]
 [ 3.00342292e-01  5.21485686e-01 -2.31038872e-02]
 [ 3.13619256e-01  4.62595075e-01 -5.16626760e-02]
 [ 3.42392623e-01  4.75814790e-01 -7.07797408e-02]
 [ 3.63148153e-01  4.99060839e-01 -8.19202513e-02]
 [ 2.83356547e-01  5.35343230e-01 -3.55235524e-02]
 [ 3.02717388e-01  4.76103216e-01 -6.66719005e-02]
 [ 3.36917371e-01  4.95128512e-01 -8.29909816e-02]
 [ 3.59486759e-01  5.20115793e-01 -9.14982259e-02]
 [ 2.71865129e-01  5.61234355e-01 -4.93730754e-02]
 [ 2.87446558e-01  5.13220668e-01 -7.64246583e-02]
 [ 3.24817181e-01  5.21417201e-01 -8.54660496e-02]
 [ 3.52097124e-01  5.37280917e-01 -8.83627906e-02]
 [ 2.65781224e-01  5.95876217e-01 -6.38981014e-02]
 [ 2.82158911e-01  5.63683629e-01 -8.30524191e-02]
 [ 3.09740096e-01  5.63894570e-01 -8.91322643e-02]
 [ 3.32126439e-01  5.71368814e-01 -9.18273479e-02]]
FLATTEN RIGHT HAND
[ 2.81015396e-01  6.84284389e-01  2.25725302e-07  3.25042158e-01
  6.32034183e-01 -5.77157876e-03  3.48924816e-01  5.87240875e-01
 -1.95303522e-02  3.74168038e-01  5.62958658e-01 -3.58712152e-02
  4.00897473e-01  5.43651700e-01 -5.09248823e-02  3.00342292e-01
  5.21485686e-01 -2.31038872e-02  3.13619256e-01  4.62595075e-01
 -5.16626760e-02  3.42392623e-01  4.75814790e-01 -7.07797408e-02
  3.63148153e-01  4.99060839e-01 -8.19202513e-02  2.83356547e-01
  5.35343230e-01 -3.55235524e-02  3.02717388e-01  4.76103216e-01
 -6.66719005e-02  3.36917371e-01  4.95128512e-01 -8.29909816e-02
  3.59486759e-01  5.20115793e-01 -9.14982259e-02  2.71865129e-01
  5.61234355e-01 -4.93730754e-02  2.87446558e-01  5.13220668e-01
 -7.64246583e-02  3.24817181e-01  5.21417201e-01 -8.54660496e-02
  3.52097124e-01  5.37280917e-01 -8.83627906e-02  2.65781224e-01
  5.95876217e-01 -6.38981014e-02  2.82158911e-01  5.63683629e-01
 -8.30524191e-02  3.09740096e-01  5.63894570e-01 -8.91322643e-02
  3.32126439e-01  5.71368814e-01 -9.18273479e-02]

POSE
[[ 0.34277141  0.37103415 -0.60591573]
 [ 0.36245558  0.3238591  -0.54652113]
 [ 0.37275884  0.32508728 -0.54651481]
 [ 0.38684601  0.3276161  -0.54648948]
 [ 0.32096955  0.32383564 -0.54488492]
 [ 0.30362636  0.32550681 -0.54434675]
 [ 0.28881273  0.32846424 -0.5444783 ]
 [ 0.40166539  0.36309397 -0.21585211]
 [ 0.26400247  0.36292601 -0.1872105 ]
 [ 0.36657071  0.43327087 -0.48811173]
 [ 0.31054315  0.4356491  -0.48097104]
 [ 0.51925749  0.68213928 -0.061697  ]
 [ 0.17186542  0.66107279 -0.21649845]
 [ 0.57232499  0.98856103 -0.0615256 ]
 [ 0.14059384  0.98428249 -0.76311284]
 [ 0.59573752  1.32562709 -0.13918172]
 [ 0.29286724  0.67308211 -1.47496474]
 [ 0.62463558  1.41173613 -0.17665464]
 [ 0.32802913  0.57385314 -1.62750471]
 [ 0.59074199  1.40170217 -0.19715476]
 [ 0.32359305  0.55000848 -1.58009303]
 [ 0.56747198  1.37243795 -0.15908986]
 [ 0.31660581  0.58421338 -1.47735202]
 [ 0.47540841  1.35055733 -0.03848253]
 [ 0.21527162  1.35407138  0.04255222]
 [ 0.45890445  1.90618944 -0.00566782]
 [ 0.22401869  1.91260719  0.09190618]
 [ 0.464075    2.3885951   0.54453987]
 [ 0.24773411  2.39138699  0.50195372]
 [ 0.46809873  2.45798635  0.56597739]
 [ 0.24533752  2.45635605  0.53352463]
 [ 0.4464159   2.54198766  0.06828859]
 [ 0.28762868  2.55611205 -0.01364078]]
FLATTEN POSE
[ 0.34277141  0.37103415 -0.60591573  0.36245558  0.3238591  -0.54652113
  0.37275884  0.32508728 -0.54651481  0.38684601  0.3276161  -0.54648948
  0.32096955  0.32383564 -0.54488492  0.30362636  0.32550681 -0.54434675
  0.28881273  0.32846424 -0.5444783   0.40166539  0.36309397 -0.21585211
  0.26400247  0.36292601 -0.1872105   0.36657071  0.43327087 -0.48811173
  0.31054315  0.4356491  -0.48097104  0.51925749  0.68213928 -0.061697
  0.17186542  0.66107279 -0.21649845  0.57232499  0.98856103 -0.0615256
  0.14059384  0.98428249 -0.76311284  0.59573752  1.32562709 -0.13918172
  0.29286724  0.67308211 -1.47496474  0.62463558  1.41173613 -0.17665464
  0.32802913  0.57385314 -1.62750471  0.59074199  1.40170217 -0.19715476
  0.32359305  0.55000848 -1.58009303  0.56747198  1.37243795 -0.15908986
  0.31660581  0.58421338 -1.47735202  0.47540841  1.35055733 -0.03848253
  0.21527162  1.35407138  0.04255222  0.45890445  1.90618944 -0.00566782
  0.22401869  1.91260719  0.09190618  0.464075    2.3885951   0.54453987
  0.24773411  2.39138699  0.50195372  0.46809873  2.45798635  0.56597739
  0.24533752  2.45635605  0.53352463  0.4464159   2.54198766  0.06828859
  0.28762868  2.55611205 -0.01364078]

RESULT ALL
[array([ 0.34012839,  0.39028478, -0.02805694, ...,  0.38656992,
        0.30880308,  0.02275069]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([ 2.81015396e-01,  6.84284389e-01,  2.25725302e-07,  3.25042158e-01,
        6.32034183e-01, -5.77157876e-03,  3.48924816e-01,  5.87240875e-01,
       -1.95303522e-02,  3.74168038e-01,  5.62958658e-01, -3.58712152e-02,
        4.00897473e-01,  5.43651700e-01, -5.09248823e-02,  3.00342292e-01,
        5.21485686e-01, -2.31038872e-02,  3.13619256e-01,  4.62595075e-01,
       -5.16626760e-02,  3.42392623e-01,  4.75814790e-01, -7.07797408e-02,
        3.63148153e-01,  4.99060839e-01, -8.19202513e-02,  2.83356547e-01,
        5.35343230e-01, -3.55235524e-02,  3.02717388e-01,  4.76103216e-01,
       -6.66719005e-02,  3.36917371e-01,  4.95128512e-01, -8.29909816e-02,
        3.59486759e-01,  5.20115793e-01, -9.14982259e-02,  2.71865129e-01,
        5.61234355e-01, -4.93730754e-02,  2.87446558e-01,  5.13220668e-01,
       -7.64246583e-02,  3.24817181e-01,  5.21417201e-01, -8.54660496e-02,
        3.52097124e-01,  5.37280917e-01, -8.83627906e-02,  2.65781224e-01,
        5.95876217e-01, -6.38981014e-02,  2.82158911e-01,  5.63683629e-01,
       -8.30524191e-02,  3.09740096e-01,  5.63894570e-01, -8.91322643e-02,
        3.32126439e-01,  5.71368814e-01, -9.18273479e-02]), array([ 0.34277141,  0.37103415, -0.60591573,  0.36245558,  0.3238591 ,
       -0.54652113,  0.37275884,  0.32508728, -0.54651481,  0.38684601,
        0.3276161 , -0.54648948,  0.32096955,  0.32383564, -0.54488492,
        0.30362636,  0.32550681, -0.54434675,  0.28881273,  0.32846424,
       -0.5444783 ,  0.40166539,  0.36309397, -0.21585211,  0.26400247,
        0.36292601, -0.1872105 ,  0.36657071,  0.43327087, -0.48811173,
        0.31054315,  0.4356491 , -0.48097104,  0.51925749,  0.68213928,
       -0.061697  ,  0.17186542,  0.66107279, -0.21649845,  0.57232499,
        0.98856103, -0.0615256 ,  0.14059384,  0.98428249, -0.76311284,
        0.59573752,  1.32562709, -0.13918172,  0.29286724,  0.67308211,
       -1.47496474,  0.62463558,  1.41173613, -0.17665464,  0.32802913,
        0.57385314, -1.62750471,  0.59074199,  1.40170217, -0.19715476,
        0.32359305,  0.55000848, -1.58009303,  0.56747198,  1.37243795,
       -0.15908986,  0.31660581,  0.58421338, -1.47735202,  0.47540841,
        1.35055733, -0.03848253,  0.21527162,  1.35407138,  0.04255222,
        0.45890445,  1.90618944, -0.00566782,  0.22401869,  1.91260719,
        0.09190618,  0.464075  ,  2.3885951 ,  0.54453987,  0.24773411,
        2.39138699,  0.50195372,  0.46809873,  2.45798635,  0.56597739,
        0.24533752,  2.45635605,  0.53352463,  0.4464159 ,  2.54198766,
        0.06828859,  0.28762868,  2.55611205, -0.01364078])]